# TODO
* add a market index for qlib models
* write prediction process for kaggle under qlib framework

# imports

In [1]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.
import sys, site
from pathlib import Path
# scripts_dir = Path.cwd().parent.joinpath("scripts")
scripts_dir = Path("/mnt/d/code/quant/qlib/scripts")
print(scripts_dir.joinpath("get_data.py"))
assert scripts_dir.joinpath("get_data.py").exists()
import numpy as np
import pandas as pd

/mnt/d/code/quant/qlib/scripts/get_data.py


In [2]:
sys.version

'3.8.13 (default, Mar 28 2022, 11:38:47) \n[GCC 7.5.0]'

In [3]:
################################# NOTE #################################
#  Please be aware that if colab installs the latest numpy and pyqlib  #
#  in this cell, users should RESTART the runtime in order to run the  #
#  following cells successfully.                                       #
########################################################################

try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    # reload
    site.main()

# scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests
    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py") as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)

In [4]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict


In [5]:
sys.path.append("/mnt/d/code/quant/qlib/scripts")
print(sys.path)

['/mnt/d/code/learn/pylearn/qlibs', '/home/jeffye/anaconda3/envs/py38/lib/python38.zip', '/home/jeffye/anaconda3/envs/py38/lib/python3.8', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/lib-dynload', '', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/wheel-0.37.1-py3.8.egg', '/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/yelib-0.1.0-py3.8.egg', '/mnt/d/code/quant/qlib/scripts']


In [6]:
from dump_bin import DumpDataUpdate

In [7]:
from data_collector.base import BaseCollector, BaseNormalize, BaseRun, Normalize
from data_collector.utils import (
    deco_retry,
    get_calendar_list,
    get_hs_stock_symbols,
    get_us_stock_symbols,
    get_in_stock_symbols,
    get_br_stock_symbols,
    generate_minutes_calendar_from_daily,
)

# explore kaggle jpx data
* "start_time": "2017-01-04", "end_time": "2022-02-28",

In [8]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data_cn --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "~/.qlib/qlib_data/kaggle"  # target_dir
# if not exists_qlib_data(provider_uri):
#     print(f"Qlib data is not found in {provider_uri}")
#     sys.path.append(str(scripts_dir))
#     from get_data import GetData
#     GetData().qlib_data(target_dir=provider_uri, region=REG_CN)
qlib.init(provider_uri=provider_uri, region=REG_CN)

[14480:MainThread](2022-05-30 20:18:16,096) INFO - qlib.Initialization - [config.py:402] - default_conf: client.
[14480:MainThread](2022-05-30 20:18:16,101) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[14480:MainThread](2022-05-30 20:18:16,102) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/jeffye/.qlib/qlib_data/kaggle')}


In [9]:
market = "all"
benchmark = "1301"

In [10]:
from qlib.data import D
# D.calendar(start_time='2010-01-01', end_time='2017-12-31', freq='day')[:2]
from qlib.data.filter import ExpressionDFilter
expressionDFilter = ExpressionDFilter(rule_expression='$close>100')
instruments = D.instruments(market='all', filter_pipe=[expressionDFilter])
# D.list_instruments(instruments=instruments, start_time='2017-01-04', end_time='2022-02-28', as_list=True)

In [11]:
instruments = ['1301', '1332']
fields = ['$close', '$volume', '$factor', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
f_d = D.features(instruments, fields, start_time='2017-01-04', end_time='2019-02-04', freq='day')
# f_d.isna().sum()
f_d

$close    $volume  $factor  Ref($close, 1)  \
instrument datetime                                                 
1301       2017-01-04  2742.0    31400.0      1.0             NaN   
           2017-01-05  2738.0    17900.0      1.0          2742.0   
           2017-01-06  2740.0    19900.0      1.0          2738.0   
           2017-01-10  2748.0    24200.0      1.0          2740.0   
           2017-01-11  2745.0     9300.0      1.0          2748.0   
...                       ...        ...      ...             ...   
1332       2019-01-29   675.0  1921600.0      1.0           663.0   
           2019-01-30   672.0  1520200.0      1.0           675.0   
           2019-01-31   670.0  1595700.0      1.0           672.0   
           2019-02-01   665.0  1420600.0      1.0           670.0   
           2019-02-04   673.0  1617900.0      1.0           665.0   

                       Mean($close, 3)  $high-$low  
instrument datetime                                 
1301       2017-01-04      2742.000000        25.0  
           2017-01-05      2740.000000        12.0  
           2017-01-06      2740.000000        24.0  
           2017-01-10      2742.000000        19.0  
           2017-01-11      2744.333252        15.0  
...                                ...         ...  
1332       2019-01-29       670.000000        12.0  
           2019-01-30       670.000000        11.0  
           2019-01-31       672.333313        15.0  
           2019-02-01       669.000000        12.0  
           2019-02-04       669.333313        16.0  

[1026 rows x 6 columns]

In [12]:
cols = fields
df =f_d
df[cols].groupby("datetime").rank(pct=True)

$close  $volume  $factor  Ref($close, 1)  \
instrument datetime                                               
1301       2017-01-04     1.0      0.5     0.75             NaN   
           2017-01-05     1.0      0.5     0.75             1.0   
           2017-01-06     1.0      0.5     0.75             1.0   
           2017-01-10     1.0      0.5     0.75             1.0   
           2017-01-11     1.0      0.5     0.75             1.0   
...                       ...      ...      ...             ...   
1332       2019-01-29     0.5      1.0     0.75             0.5   
           2019-01-30     0.5      1.0     0.75             0.5   
           2019-01-31     0.5      1.0     0.75             0.5   
           2019-02-01     0.5      1.0     0.75             0.5   
           2019-02-04     0.5      1.0     0.75             0.5   

                       Mean($close, 3)  $high-$low  
instrument datetime                                 
1301       2017-01-04              1.0         1.0  
           2017-01-05              1.0         1.0  
           2017-01-06              1.0         1.0  
           2017-01-10              1.0         1.0  
           2017-01-11              1.0         1.0  
...                                ...         ...  
1332       2019-01-29              0.5         0.5  
           2019-01-30              0.5         0.5  
           2019-01-31              0.5         0.5  
           2019-02-01              0.5         0.5  
           2019-02-04              0.5         0.5  

[1026 rows x 6 columns]

## remove 60 days for Alpha360 handler

In [13]:
df = f_d
df.index = df.index.get_level_values('datetime')
print(df.index.min(), df.index.max())


start_time = pd.to_datetime(df.index.min()) + pd.Timedelta(days=60)
print(start_time.strftime('%Y-%m-%d'))

2017-01-04 00:00:00 2019-02-04 00:00:00
2017-03-05


# train model

## define static parameters

In [14]:
experiment_name="online_srv"

In [15]:
###################################
# train model
###################################
# '2017-01-04', end_time='2022-02-28'
data_handler_config = {
    # "start_time": "2017-01-04",
    "start_time": start_time, # 
    "end_time": "2019-02-04",
    "fit_start_time": "2017-01-04",
    "fit_end_time": "2018-07-31",
    "instruments": market,
    "infer_processors": [
      {
        "class": "RobustZScoreNorm",
        "kwargs": {
          "fields_group": "feature",
          "clip_outlier": True
        }
      },
      {
        "class": "Fillna",
        "kwargs": {
          "fields_group": "feature"
        }
      }
    ],
    "learn_processors": [
      {
        "class": "DropnaLabel"
      },
      {
        "class": "CSRankNorm",
        "kwargs": {
          "fields_group": "label"
        }
      }
    ],
    "label": [
      "Ref($close, -2) / Ref($close, -1) - 1"
    ]
}

task = {   
    "model": {
        "class": "LSTM",
        "module_path": "qlib.contrib.model.pytorch_lstm",
        "kwargs": {
            "d_feat": 6,
            "hidden_size": 64,
            "num_layers": 2,
            "dropout": 0.1,
            "dec_dropout": 0.0,
            "n_epochs": 15,
            "lr": 1e-4,
            "early_stop": 3,
            "batch_size": 800,
            "metric": "loss",
            "loss": "mse",
            "optimizer": "adam",
            "GPU": 1
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha360",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": (start_time, "2018-07-31"),
                "valid": ("2018-08-01", "2019-02-04"),
                "test": ("2018-08-01", "2019-02-04"),
            },
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])


Please install necessary libs for CatBoostModel.


/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[14480:MainThread](2022-05-30 20:18:21,072) INFO - qlib.LSTM - [pytorch_lstm.py:58] - LSTM pytorch version...
[14480:MainThread](2022-05-30 20:18:21,375) INFO - qlib.LSTM - [pytorch_lstm.py:75] - LSTM parameters setting:
d_feat : 6
hidden_size : 64
num_layers : 2
dropout : 0.1
n_epochs : 15
lr : 0.0001
metric : loss
batch_size : 800
early_stop : 3
optimizer : adam
loss_type : mse
visible_GPU : 1
use_GPU : True
seed : None
[14480:MainThread](2022-05-30 20:19:19,309) INFO - qlib.timer - [log.py:113] - Time cost: 49.445s | Loading data Done
/home/jeffye/anaconda3/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1096: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian

## explore Ahpha360 (DataHandlerLP) for data processing

In [ ]:
dataset.handler._infer.iloc[-100].to_string()

In [17]:
# dataset.handler._learn.iloc[-20:]
pd.set_option('display.max_rows', 400)
na_sum = dataset.handler._learn.isna().sum()

In [ ]:
na_sum.to_string()

In [19]:
dataset.handler._learn.loc[:, pd.IndexSlice[:, ['CLOSE0', 'OPEN0', 'LOW0', 'VOLUME0', 'HIGH0']]]

feature                                      
                       CLOSE0     OPEN0      LOW0 VOLUME0     HIGH0
datetime   instrument                                              
2017-03-06 1301           0.0 -1.647025 -1.495574     0.0 -0.389995
           1332           0.0  2.003291  0.574387     0.0  2.558551
           1333           0.0 -0.488532 -0.082080     0.0 -0.839524
           1376           0.0  0.348450  0.691736     0.0 -0.488260
           1377           0.0  0.663846  0.640347     0.0  0.001264
...                       ...       ...       ...     ...       ...
2019-02-04 9990           0.0 -0.922927 -0.376043     0.0  1.238996
           9991           0.0 -2.371414 -3.000000     0.0 -0.521415
           9993           0.0 -0.147190  0.823325     0.0  0.722399
           9994           0.0 -2.001480 -2.281639     0.0 -0.610913
           9997           0.0 -0.964642 -0.887971     0.0  1.965344

[890224 rows x 5 columns]

In [30]:
# dataset.handler._data.iloc[-20:].isna().sum().to_string()

# learn_ processors
csrank_norm = dataset.handler.learn_processors[1]

from qlib.data.dataset.processor import get_group_columns


print(csrank_norm.fields_group)
# cols = get_group_columns(df, csrank_norm.fields_group)
# print(cols)

label


In [33]:
# infer processor
robust_zscorenorm = dataset.handler.infer_processors[0]

_data = dataset.handler._data.copy()
print(robust_zscorenorm.fields_group)

feature


In [42]:
# X = _data[robust_zscorenorm.cols]
# X -= robust_zscorenorm.mean_train
# X /= robust_zscorenorm.std_train
# X.clip(-3, 3).iloc[:10]

_shared_df = dataset.handler._data
_shared_df = _shared_df.copy()
# 2) process
# _shared_df = dataset.handler._run_proc_l(_shared_df, dataset.handler.shared_processors, with_fit=True)

# data for inference
# 1) assign
_infer_df = _shared_df
if not dataset.handler._is_proc_readonly(dataset.handler.infer_processors):  # avoid modifying the original data
    _infer_df = _infer_df.copy()
# 2) process
_infer_df = dataset.handler._run_proc_l(_infer_df, dataset.handler.infer_processors, with_fit=True, check_for_infer=True)
_infer_df

feature                                                  \
                      CLOSE59 CLOSE58 CLOSE57 CLOSE56 CLOSE55 CLOSE54 CLOSE53   
datetime   instrument                                                           
2017-03-06 1301           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1332           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1333           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1376           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1377           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1379           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1381           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1407           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1414           NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           1417           NaN     NaN     NaN     NaN     NaN     NaN     NaN   

                                               ...                      \
                      CLOSE52 CLOSE51 CLOSE50  ...   VOLUME8   VOLUME7   
datetime   instrument                          ...                       
2017-03-06 1301           NaN     NaN     NaN  ...  0.464621  0.256607   
           1332           NaN     NaN     NaN  ...  1.748983  0.931908   
           1333           NaN     NaN     NaN  ...  0.924786  1.115385   
           1376           NaN     NaN     NaN  ...  1.593750  1.031250   
           1377           NaN     NaN     NaN  ...  1.131016  0.852941   
           1379           NaN     NaN     NaN  ...  1.287532  1.167939   
           1381           NaN     NaN     NaN  ...  1.562500  1.250000   
           1407           NaN     NaN     NaN  ...  0.917284  1.076543   
           1414           NaN     NaN     NaN  ...  3.085106  1.051064   
           1417           NaN     NaN     NaN  ...  0.866356  0.620047   

                                                                         \
                        VOLUME6   VOLUME5   VOLUME4   VOLUME3   VOLUME2   
datetime   instrument                                                     
2017-03-06 1301        0.276215  0.439898  0.387042  0.656437  0.447570   
           1332        1.002992  0.964443  0.811004  1.138450  0.887491   
           1333        0.932479  0.905983  1.027778  0.842308  1.033761   
           1376        1.187500  1.718750  1.687500  1.125000  0.968750   
           1377        1.411765  1.700535  1.467914  1.442513  1.550802   
           1379        1.109415  1.089059  1.786260  1.394402  1.938931   
           1381        0.968750  0.531250  0.406250  0.125000  2.156250   
           1407        0.802469  1.096296  1.275309  1.253086  1.023457   
           1414        1.304255  1.910638  2.263830  2.317021  2.487234   
           1417        0.828283  1.201632  0.943279  1.231546  1.401709   

                                                                        label  
                        VOLUME1 VOLUME0 Ref($close, -2) / Ref($close, -1) - 1  
datetime   instrument                                                          
2017-03-06 1301        0.385337     1.0                              0.006420  
           1332        0.986666     1.0                             -0.003650  
           1333        1.194017     1.0                             -0.002861  
           1376        1.656250     1.0                             -0.006873  
           1377        1.391711     1.0                              0.009231  
           1379        1.262087     1.0                             -0.002740  
           1381        0.906250     1.0                             -0.004000  
           1407        1.474074     1.0                              0.008631  
           1414        2.104255     1.0                              0.006079  
           1417        1.387723     1.0                              

In [80]:
processor = dataset.handler.infer_processors
processor
# _infer_df = _shared_df.copy()
# _infer_df.iloc[:10]
dataset.handler._data.loc["2019-02-04"].iloc[-10:]

feature                                                    \
             CLOSE59   CLOSE58   CLOSE57   CLOSE56   CLOSE55   CLOSE54   
instrument                                                               
9982        1.107771  1.095292  1.104935  1.096426  1.108905  1.108338   
9983        1.197665  1.140681  1.156312  1.196082  1.208350  1.192917   
9984        1.029224  1.034903  1.014553  1.012423  1.041765  1.039399   
9987        0.987931  0.987931  1.025862  1.046552  1.056897  1.044828   
9989        1.159888  1.152875  1.171108  1.150070  1.178121  1.180926   
9990        0.996396  0.990090  0.990090  0.983784  0.990991  1.015315   
9991        1.038314  1.040230  1.053640  1.047893  1.065134  1.070881   
9993        0.928161  0.928161  0.927586  0.928736  0.930460  0.945402   
9994        1.208296  1.204238  1.199279  1.193418  1.214608  1.206943   
9997        1.222599  1.291525  1.265537  1.282486  1.325424  1.327684   

                                                    ...                      \
             CLOSE53   CLOSE52   CLOSE51   CLOSE50  ...   VOLUME8   VOLUME7   
instrument                                          ...                       
9982        1.098695  1.094158  1.089053  1.091889  ...  0.254144  0.475138   
9983        1.203007  1.189553  1.167392  1.173526  ...  2.137892  1.962599   
9984        1.038452  1.057856  1.107667  1.077733  ...  1.541432  0.852809   
9987        1.032759  1.013793  1.027586  1.024138  ...  1.295606  1.133600   
9989        1.179523  1.158485  1.141655  1.140252  ...  2.359252  1.925256   
9990        0.995495  0.987387  0.988288  0.985586  ...  0.597892  1.359940   
9991        1.067050  1.045019  1.034483  1.039272  ...  0.230769  0.337469   
9993        0.971839  0.967816  0.969540  0.979885  ...  0.607843  1.627451   
9994        1.102795  1.154193  1.110009  1.128494  ...  0.492647  0.327206   
9997        1.348023  1.323164  1.346893  1.332203  ...  0.483684  0.410684   

                                                                        \
             VOLUME6   VOLUME5   VOLUME4   VOLUME3   VOLUME2   VOLUME1   
instrument                                                               
9982        0.408840  0.408840  0.751381  0.530387  0.552486  1.331492   
9983        1.975633  1.116547  1.459955  1.129581  1.408387  1.836985   
9984        0.978865  0.767552  1.126911  1.010345  1.841951  1.127458   
9987        1.018198  0.678207  0.813582  1.088771  1.460719  1.122059   
9989        1.598553  1.194093  2.958409  1.881857  2.025316  1.086799   
9990        1.040663  0.384036  0.667169  1.067771  1.302711  0.745482   
9991        0.265509  0.377171  0.265509  0.774194  1.429280  1.062035   
9993        1.686275  0.352941  0.392157  1.019608  0.568627  0.431373   
9994        1.000000  0.386029  0.672794  0.768382  0.397059  0.621324   
9997        0.335992  0.493594  0.488760  0.647571  0.693498  1.940295   

                                                   label  
           VOLUME0 Ref($close, -2) / Ref($close, -1) - 1  
instrument                                                
9982           1.0                             -0.001675  
9983           1.0                              0.005092  
9984           1.0                              0.005466  
9987           1.0                              0.000000  
9989           1.0                             -0.008415  
9990           1.0                             -0.012635  
9991           1.0                             -0.014327  
9993           1.0                              0.007995  
9994           1.0                              0.006685  
9997           1.0                              0.030892  

[10 rows x 361 columns]

In [104]:
cols = ['CLOSE0', 'OPEN0', 'HIGH0', 'LOW0', 'VWAP0', 'VOLUME0']
t_temp = dataset.handler._data.loc[:, (['feature'], cols)]

t_temp
# print(t_temp.isna().sum())
# print(t_temp.sum())

feature                                            
                       CLOSE0     OPEN0     HIGH0      LOW0 VWAP0 VOLUME0
datetime   instrument                                                    
2017-03-06 1301           1.0  0.980707  1.004823  0.980707   NaN     1.0
           1332           1.0  1.023466  1.027076  0.996390   NaN     1.0
           1333           1.0  0.994278  1.001431  0.991416   NaN     1.0
           1376           1.0  1.004082  1.004082  0.997279   NaN     1.0
           1377           1.0  1.007776  1.007776  0.996890   NaN     1.0
...                       ...       ...       ...       ...   ...     ...
2019-02-04 9990           1.0  0.989189  1.017117  0.989189   NaN     1.0
           9991           1.0  0.972222  1.003831  0.968391   NaN     1.0
           9993           1.0  0.998276  1.013218  0.998276   NaN     1.0
           9994           1.0  0.976555  1.003156  0.974752   NaN     1.0
           9997           1.0  0.988701  1.022599  0.985311   NaN     1.0

[893587 rows x 6 columns]

In [113]:
t_temp = dataset.handler._data.loc["2019-02-04"]
cols = ['CLOSE4', 'CLOSE3', 'CLOSE2', 'CLOSE1', 'CLOSE0']
# cols = ['VOLUME4', 'VOLUME3','VOLUME2','VOLUME1','VOLUME0']

t1 = t_temp.loc[:, (['feature'], cols)]
print('shape', t1.shape)

nan_rows = t1[pd.isnull(t1[('feature', cols[1])])]

print(nan_rows.isna().sum())
print(nan_rows)

insts = ['6824', '9535', '2790', '9982', '2226']

t1.isna().sum()
t1.loc[insts]


shape (1930, 5)
feature  CLOSE4     4
         CLOSE3    11
         CLOSE2     8
         CLOSE1     5
         CLOSE0     4
dtype: int64
             feature                                  
              CLOSE4 CLOSE3    CLOSE2    CLOSE1 CLOSE0
instrument                                            
1981             NaN    NaN       NaN       NaN    NaN
2226        1.011179    NaN  1.008130  1.013211    1.0
2761        0.958730    NaN       NaN  0.976190    1.0
2923        0.976303    NaN  0.969984  0.982622    1.0
4365             NaN    NaN       NaN       NaN    NaN
4628        1.023810    NaN       NaN  1.015152    1.0
4781             NaN    NaN       NaN       NaN    NaN
4966        1.031496    NaN       NaN  1.006299    1.0
5945             NaN    NaN       NaN       NaN    NaN
9539        1.000342    NaN       NaN       NaN    1.0
9640        0.960279    NaN  0.977003  0.977700    1.0


feature                                     
              CLOSE4    CLOSE3    CLOSE2    CLOSE1 CLOSE0
instrument                                               
6824        0.988201  0.997050  0.985251       NaN    1.0
9535        1.014793  1.005917  1.002959  0.991124    1.0
2790        1.001178  0.978210       NaN       NaN    1.0
9982        1.007941  1.007941  0.996597  0.972774    1.0
2226        1.011179       NaN  1.008130  1.013211    1.0

In [114]:
t_temp = dataset.handler._infer.loc["2019-02-04"]
t1 = t_temp.loc[:, (['feature'], ['CLOSE4', 'CLOSE3', 'CLOSE2', 'CLOSE1', 'CLOSE0'])]

# insts = ['6824', '9535', '2790', '9982']

t1.isna().sum()
t1.loc[insts]


feature                                     
              CLOSE4    CLOSE3    CLOSE2    CLOSE1 CLOSE0
instrument                                               
6824       -0.353115 -0.069895 -0.726091  0.000000    0.0
9535        0.585218  0.298132  0.187378 -0.673491    0.0
2790        0.104800 -0.851872  0.000000  0.000000    0.0
9982        0.343442  0.382131 -0.140802 -2.065937    0.0
2226        0.457693  0.000000  0.454155  1.002480    0.0

In [115]:
t_temp = dataset.handler._infer.loc["2019-02-04"]
cols = ['CLOSE0', 'OPEN0', 'HIGH0', 'LOW0', 'VWAP0', 'VOLUME0']
t1 = t_temp.loc[:, (['feature'], cols)]

t1.loc[insts]

feature                                            
            CLOSE0     OPEN0     HIGH0      LOW0 VWAP0 VOLUME0
instrument                                                    
6824           0.0 -0.251830 -0.638224  0.661541   0.0     0.0
9535           0.0  0.505156 -0.245038  0.269881   0.0     0.0
2790           0.0  0.000000 -1.029088  1.050896   0.0     0.0
9982           0.0 -2.324335  2.954295 -2.542764   0.0     0.0
2226           0.0  1.561660  1.394757  0.514343   0.0     0.0

In [116]:
print(dataset.handler._data.isna().sum().sum())
print(dataset.handler._data.shape)

56447823
(893587, 361)


In [117]:
56447823/(893587 * 361)

0.17498593943826393

In [ ]:
# start exp to train model

experiment_id = 'jpx'
# experiment_name: Optional[Text] = None,
# recorder_id: Optional[Text] = None,

# with R.start(experiment_name=experiment_name, experiment_id=experiment_id):
with R.start(experiment_name=experiment_name):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id

In [22]:
print(rid, experiment_name)

db46842656cf42a7859de5ba753f4ea6 online_srv


# prediction, backtest & analysis

In [23]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 200,
            "n_drop": 20,
        },
    },
    "backtest": {
        "start_time": "2021-12-06",
        "end_time": "2022-02-27",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
backtest_experiment_name = "backtest1"
with R.start(experiment_name=backtest_experiment_name, experiment_id='back0'): 
    recorder = R.get_recorder(recorder_id=rid, experiment_name=experiment_name)
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # # backtest & analysis
    # par = PortAnaRecord(recorder, port_analysis_config, "day")
    # par.generate()


[14480:MainThread](2022-05-30 20:25:18,752) INFO - qlib.workflow - [expm.py:315] - <mlflow.tracking.client.MlflowClient object at 0x7f3225a98a00>
[14480:MainThread](2022-05-30 20:25:18,764) WARNING - qlib.workflow - [expm.py:195] - No valid experiment found. Create a new experiment with name backtest1.


ExpAlreadyExistError: 

In [ ]:
# 模型的predict score，ADD模型： ADDModel.forward() 输出"excess"，["market"]，["adv_market"]，["adv_excess"]，"reconstructed_feature"
# ADD.predict() 取的是“excess”, 成为pred.pkl 中的score.

# SignalRecord中实现如下：
def generate(self, **kwargs):
    # generate prediciton
    pred = self.model.predict(self.dataset)
    if isinstance(pred, pd.Series):
        pred = pred.to_frame("score")
    self.save(**{"pred.pkl": pred})

    logger.info(
        f"Signal record 'pred.pkl' has been saved as the artifact of the Experiment {self.recorder.experiment_id}"
    )
    # print out results
    pprint(f"The following are prediction results of the {type(self.model).__name__} model.")
    pprint(pred.head(5))

    if isinstance(self.dataset, DatasetH):
        raw_label = self.generate_label(self.dataset)
        self.save(**{"label.pkl": raw_label})

pred_score = par.load("pred.pkl")
pred_score.head(2)

NameError: name 'par' is not defined

In [ ]:
s = pd.Series(["a", "b", "c"], name='vals')
print(s)
s.to_frame()

# analyze graphs

In [ ]:
from pprint import pprint

import qlib
import pandas as pd
from qlib.utils.time import Freq
from qlib.utils import flatten_dict
from qlib.contrib.evaluate import backtest_daily
from qlib.contrib.evaluate import risk_analysis
from qlib.contrib.strategy import TopkDropoutStrategy

CSI300_BENCH = benchmark
STRATEGY_CONFIG = {
    "topk": 50,
    "n_drop": 5,
    # pred_score, pd.Series
    "signal": pred_score,
}


strategy_obj = TopkDropoutStrategy(**STRATEGY_CONFIG)
report_normal, positions_normal = backtest_daily(
    start_time="2021-12-06", end_time="2022-02-27", strategy=strategy_obj
)
analysis = dict()
# default frequency will be daily (i.e. "day")
analysis["excess_return_without_cost"] = risk_analysis(report_normal["return"] - report_normal["bench"])
analysis["excess_return_with_cost"] = risk_analysis(report_normal["return"] - report_normal["bench"] - report_normal["cost"])

analysis_df = pd.concat(analysis)  # type: pd.DataFrame
pprint(analysis_df)

In [ ]:
# report_normal, positions_normal = backtest_daily(
#     start_time="2017-01-01", end_time="2020-08-01", strategy=strategy_obj
# )
from typing import Union
from qlib.backtest import backtest
from qlib.backtest.executor import SimulatorExecutor

start_time = "2017-01-01"
end_time = "2020-08-01"
strategy = strategy_obj
executor = None
account = 1.e8
benchmark: str = benchmark
exchange_kwargs: dict = None
pos_type: str = "Position"

freq = "day"
if executor is None:
    executor_config = {
        "time_per_step": freq,
        "generate_portfolio_metrics": True,
    }
    executor = SimulatorExecutor(**executor_config)
    
_exchange_kwargs = {
    "freq": freq,
    "limit_threshold": None,
    "deal_price": None,
    "open_cost": 0.0005,
    "close_cost": 0.0015,
    "min_cost": 5,
}
# if exchange_kwargs is not None:
#     _exchange_kwargs.update(exchange_kwargs)

print(account)

portfolio_metric_dict, indicator_dict = backtest(
    start_time=start_time,
    end_time=end_time,
    strategy=strategy,
    executor=executor,
    account=account,
    benchmark=benchmark,
    exchange_kwargs=_exchange_kwargs,
    pos_type=pos_type,
)



# analysis_freq = "{0}{1}".format(*Freq.parse(freq))

# report_normal, positions_normal = portfolio_metric_dict.get(analysis_freq)

In [ ]:
strategy.signal.signal_cache.loc['2017-01-10']
# pred_score.loc['2017-01-04']
print('position type', type(strategy.common_infra.get("trade_account").current_position))
position_dict = strategy.common_infra.get("trade_account").current_position.position


print(len(position_dict), position_dict)  # 默认维护52只股票 + cash + now_account_value
# 单只股票信息：
# 'SZ000001': {'amount': 392925.71123170934, 'price': 8.12631893157959, 'weight': 0.016886521190966564, 'count_day': 55}

# executor.track_data

In [ ]:
# portfolio_metric_dict --> 计算在account类中, def _update_state_from_order(self, order, trade_val, cost, trade_price) --》AccumulatedInfo()
# report 中也记录了turnover,
# turnover 定义：https://www.investopedia.com/terms/p/portfolioturnover.asp

# type(portfolio_metric_dict)
executor.trade_account.accum_info.to

In [ ]:
# portfolio_metric_dict 的具体计算方法见：account.update_portfolio_metrics, 计算依据position 类中记录的信息qlib.backtest.position.py
# SimulatorExecutor 中维护（生成）： 1. self.level_infra = LevelInfrastructure() -->def reset_common_infra L117 生成trade_account --> 
# account中维护position 

# portfolio_metric_dict

# executor 和 strategy中使用/维护的是同一个account 和 同一个common_infra
print(executor.trade_account)
strategy.common_infra.get("trade_account")
print(strategy.common_infra, executor.common_infra)


In [ ]:
# indicator_dict 主要也在account中计算。
# ffr: order fulfill rate,  pa: order price advantage, pos:positive_rate 更多参考 BaseOrderIndicator
# class Indicator:
#     """
#     `Indicator` is implemented in a aggregate way.
#     All the metrics are calculated aggregately.
#     All the metrics are calculated for a separated stock and in a specific step on a specific level.

#     | indicator    | desc.                                                        |
#     |--------------+--------------------------------------------------------------|
#     | amount       | the *target* amount given by the outer strategy              |
#     | deal_amount  | the real deal amount                                         |
#     | inner_amount | the total *target* amount of inner strategy                  |
#     | trade_price  | the average deal price                                       |
#     | trade_value  | the total trade value                                        |
#     | trade_cost   | the total trade cost  (base price need drection)             |
#     | trade_dir    | the trading direction                                        |
#     | ffr          | full fill rate                                               |
#     | pa           | price advantage                                              |
#     | pos          | win rate                                                     |
#     | base_price   | the price of baseline                                        |
#     | base_volume  | the volume of baseline (for weighted aggregating base_price) |

#     **NOTE**:
#     The `base_price` and `base_volume` can't be NaN when there are not trading on that step. Otherwise
#     aggregating get wrong results.

#     So `base_price` will not be calculated in a aggregate way!!

#     """
indicator_dict

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)

# jpx prediction

In [118]:
import sys
sys.path.append("/mnt/d/dataset/quant/kaggle22/")

import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

<frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.7 of module 'jpx_tokyo_market_prediction.competition' does not match runtime version 3.8


In [119]:
prices, options, financials, trades, secondary_prices, sample_prediction = next(iter_test)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [ ]:
prices
start_date = prices['Date'].min()
end_date = prices['Date'].max()
print(type(start_date), start_date, end_date)

prices = prices.rename(columns={"SecuritiesCode": "instrument", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume", "AdjustmentFactor": "factor", "Date": "date"})

In [140]:
prices['date'] = pd.to_datetime(prices['date'])
prices['instrument'] = prices['instrument'].astype(str)

In [145]:
# prices.info()
prices.dtypes

date                datetime64[ns]
RowId                       object
instrument                  object
open                       float64
high                       float64
low                        float64
close                      float64
volume                       int64
factor                     float64
ExpectedDividend           float64
SupervisionFlag               bool
dtype: object

In [156]:
from dump_bin import DumpDataBase, DumpDataUpdate
from typing import Union
from loguru import logger
from qlib.utils import fname_to_code, code_to_fname

class DumpDataUpdateInMemory(DumpDataUpdate):
    def __init__(
        self,
        data : pd.DataFrame,
        csv_path: str,
        qlib_dir: str,
        backup_dir: str = None,
        freq: str = "day",
        max_workers: int = 16,
        date_field_name: str = "date",
        file_suffix: str = ".csv",
        symbol_field_name: str = "symbol",
        exclude_fields: str = "",
        include_fields: str = "",
        limit_nums: int = None,
    ):
        self.data = data
        super().__init__(
            csv_path,
            qlib_dir,
            backup_dir,
            freq,
            max_workers,
            date_field_name,
            file_suffix,
            symbol_field_name,
            exclude_fields,
            include_fields,
        )

    def _load_all_source_data(self):
        return self.data
    

    def save_instruments(self, instruments_data: Union[list, pd.DataFrame]):
        self._instruments_dir.mkdir(parents=True, exist_ok=True)
        instruments_path = str(self._instruments_dir.joinpath(self.INSTRUMENTS_FILE_NAME).resolve())
        if isinstance(instruments_data, pd.DataFrame):
            _df_fields = [self.symbol_field_name, self.INSTRUMENTS_START_FIELD, self.INSTRUMENTS_END_FIELD]
            instruments_data = instruments_data.loc[:, _df_fields]
            logger.info(f'{self.symbol_field_name}, with {instruments_data.info()}')



            instruments_data[self.symbol_field_name] = instruments_data[self.symbol_field_name].astype(str).apply(
                lambda x: fname_to_code(x.lower()).upper()
            )
            instruments_data.to_csv(instruments_path, header=False, sep=self.INSTRUMENTS_SEP, index=False)
        else:
            np.savetxt(instruments_path, instruments_data, fmt="%s", encoding="utf-8")

In [157]:
_dump = DumpDataUpdateInMemory(data=prices, csv_path='~/.', qlib_dir=provider_uri, symbol_field_name='instrument', exclude_fields="RowId")
_dump.dump()

# print(_dump.symbol_field_name)

2022-05-31 16:02:29.979 | INFO     | dump_bin:_dump_features:456 - start dump features......
100%|██████████| 2000/2000 [00:03<00:00, 545.43it/s]  
2022-05-31 16:02:40.435 | INFO     | dump_bin:_dump_features:491 - dump bin errors： {}
2022-05-31 16:02:40.498 | INFO     | dump_bin:_dump_features:493 - end of features dump.

2022-05-31 16:02:40.518 | INFO     | __main__:save_instruments:46 - instrument, with None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   instrument      4000 non-null   object
 1   start_datetime  4000 non-null   object
 2   end_datetime    4000 non-null   object
dtypes: object(3)
memory usage: 93.9+ KB


In [ ]:
import pandas as pd
from pathlib import Path
import copy

# save price to source_dir, to prepare the normalized data

source_dir = Path('~/kaggle_jpx/source').expanduser().resolve()
source_dir.mkdir(parents=True, exist_ok=True)
norm_dir = Path('~/kaggle_jpx/normalized').expanduser().resolve()
norm_dir.mkdir(parents=True, exist_ok=True)


def save_instrument(symbol, df: pd.DataFrame, save_dir, symbol_field='symbol'):
    """save instrument data to file

    Parameters
    ----------
    symbol: str
        instrument code
    df : pd.DataFrame
        df.columns must contain "symbol" and "datetime"
    """
    if df is None or df.empty:
        logger.warning(f"{symbol} is empty")
        return

    instrument_path = save_dir.joinpath(f"{symbol}.csv")
    df[symbol_field] = symbol
    if instrument_path.exists():
        _old_df = pd.read_csv(instrument_path)
        df = pd.concat([_old_df, df], sort=False)
    df.to_csv(instrument_path, index=False)



df = prices.rename(columns={"SecuritiesCode": "instrument", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume", "AdjustmentFactor": "factor", "Date": "date"})
for sc, group in df.groupby(['instrument']):
    _path = source_dir.joinpath(f"{sc}.csv")
    ret_value = group
    if _path.exists():
        _old_df = pd.read_csv(_path)
        ret_value = pd.concat([_old_df, group], sort=False)
        ret_value.drop_duplicates(subset='date', keep="last", inplace=True)
    ret_value.to_csv(_path, index=False)


In [ ]:
## 自动重新加载更改的模块
# %load_ext autoreload
# %autoreload 2

# normalize_class_name f"YahooNormalize{self.region.upper()}{self.interval}" -->YahooNormalizeCN1d
sys.path.append("/mnt/d/code/quant/qlib/scripts/data_collector/yahoo")  # in qlib source
from typing import Iterable
from loguru import logger
from data_collector.yahoo.collector import YahooNormalizeCN, YahooNormalize1d, YahooNormalize, YahooNormalize1dExtend, YahooNormalizeCN1dExtend

qlib_data_1d_dir = "~/.qlib/qlib_data/kaggle"
qlib_data_1d_dir = str(Path(qlib_data_1d_dir).expanduser().resolve())

class YahooNormalizeCN1doffline(YahooNormalizeCN, YahooNormalize1d):
    def _get_calendar_list(self) -> Iterable[pd.Timestamp]:

        import qlib
        from qlib.data import D

        qlib.init(provider_uri=qlib_data_1d_dir)
        return list(D.calendar(freq="day"))
    
    def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
        # normalize
        df = self.normalize_yahoo(df, self._calendar_list, self._date_field_name, self._symbol_field_name)
        print('normalize_yahoo', df)
        # adjusted price
        df = self.adjusted_price(df)
        print('adjusted_price', df)
        
        df = df.drop(columns=['RowId', 'ExpectedDividend', 'SupervisionFlag'])
        # print(df)
        df = self._manual_adj_data(df)
        print('_manual_adj_data', df)
        return df
    
    @staticmethod
    def normalize_yahoo(
        df: pd.DataFrame,
        calendar_list: list = None,
        date_field_name: str = "date",
        symbol_field_name: str = "symbol",
        last_close: float = None,
    ):
        input('normalize_yahoo')
        if df.empty:
            return df
        symbol = df.loc[df[symbol_field_name].first_valid_index(), symbol_field_name]
        columns = copy.deepcopy(YahooNormalize.COLUMNS)
        df = df.copy()
        df.set_index(date_field_name, inplace=True)
        df.index = pd.to_datetime(df.index)
        df = df[~df.index.duplicated(keep="first")]
        
        # print(symbol, df)
        
        if calendar_list is not None:
            df = df.reindex(
                pd.DataFrame(index=calendar_list)
                .loc[
                    pd.Timestamp(df.index.min()).date() : pd.Timestamp(df.index.max()).date()
                    + pd.Timedelta(hours=23, minutes=59)
                ]
                .index
            )
            # print('after reindex', df)
            # input("please input:")
        df.sort_index(inplace=True)
        df.loc[(df["volume"] <= 0) | np.isnan(df["volume"]), list(set(df.columns) - {symbol_field_name})] = np.nan

        change_series = YahooNormalize.calc_change(df, last_close)
        # NOTE: The data obtained by Yahoo finance sometimes has exceptions
        # WARNING: If it is normal for a `symbol(exchange)` to differ by a factor of *89* to *111* for consecutive trading days,
        # WARNING: the logic in the following line needs to be modified
        _count = 0
        while True:
            # NOTE: may appear unusual for many days in a row
            change_series = YahooNormalize.calc_change(df, last_close)
            _mask = (change_series >= 89) & (change_series <= 111)
            if not _mask.any():
                break
            _tmp_cols = ["high", "close", "low", "open", "adjclose"]
            df.loc[_mask, _tmp_cols] = df.loc[_mask, _tmp_cols] / 100
            _count += 1
            if _count >= 10:
                _symbol = df.loc[df[symbol_field_name].first_valid_index()]["symbol"]
                logger.warning(
                    f"{_symbol} `change` is abnormal for {_count} consecutive days, please check the specific data file carefully"
                )

        df["change"] = YahooNormalize.calc_change(df, last_close)

        columns += ["change"]
        df.loc[(df["volume"] <= 0) | np.isnan(df["volume"]), columns] = np.nan

        df[symbol_field_name] = symbol
        df.index.names = [date_field_name]
        return df.reset_index()
    

class YahooNormalize1dExtendFixed(YahooNormalizeCN, YahooNormalize1dExtend):
    def _get_calendar_list(self) -> Iterable[pd.Timestamp]:
        import qlib
        from qlib.data import D

        qlib.init(provider_uri=qlib_data_1d_dir)
        
        days_list = list(D.calendar(freq="day"))
        last_day = days_list[-1]
        i = 1
        while True:
            day = last_day + pd.Timedelta(days=i)
            if day < pd.Timestamp.now():
                # print(day)
                days_list.append(day)
            else:
                break
            i += 1
        
        return days_list
    
    def _get_old_data(self, qlib_data_dir: [str, Path]):
        import qlib
        from qlib.data import D

        qlib_data_dir = str(Path(qlib_data_dir).expanduser().resolve())
        qlib.init(provider_uri=qlib_data_dir, expression_cache=None, dataset_cache=None)
        df = D.features(D.instruments("all"), ["$close/$factor", "$adjclose/$close"])
        df.columns = [self._ori_close_field, self._first_close_field]
        # print(df.columns)
        # input("print columns")
        # df = df.astype({self._symbol_field_name: str,})
        return df
    
#     def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
#         df = df.astype({self._symbol_field_name: str,})
        
#         _last_close = self._get_last_close(df)
#         # reindex
#         _last_date = self._get_last_date(df)
        
        
#         df = self.normalize_yahoo(
#             df, self._calendar_list, self._date_field_name, self._symbol_field_name, last_close=_last_close
#         )
#         # adjusted price
#         df = self.adjusted_price(df)
#         df = self._manual_adj_data(df)
#         return df
        
    def print_all(self):
        print(input)
        print('iam here')
        input('still here')
        
    def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
        print('input\n', df)
        df = df.astype({self._symbol_field_name: str,})
        _last_close = self._get_last_close(df)
        # reindex
        _last_date = self._get_last_date(df)
        print('_last_date', _last_date, _last_close, self._date_field_name)
        if _last_date is not None:
            df = df.set_index(self._date_field_name)
            df.index = pd.to_datetime(df.index)
            df = df[~df.index.duplicated(keep="first")]
            _max_date = df.index.max()
            df = df.reindex(self._calendar_list).loc[:_max_date].reset_index()
            df = df[df[self._date_field_name] > _last_date]
            if df.empty:
                return pd.DataFrame()
            _si = df["close"].first_valid_index()
            if _si > df.index[0]:
                logger.warning(
                    f"{df.loc[_si][self._symbol_field_name]} missing data: {df.loc[:_si - 1][self._date_field_name].to_list()}"
                )
        
        logger.info(f'1. df={df}')
        # normalize
        df = self.normalize_yahoo(
            df, self._calendar_list, self._date_field_name, self._symbol_field_name, last_close=_last_close
        )
        
        logger.info(f'2. df={df}')
        # adjusted price
        df = self.adjusted_price(df)
        df = self._manual_adj_data(df)
        logger.info(f'3 df={df}')
        return df
        
    def _manual_adj_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """manual adjust data: All fields (except change) are standardized according to the close of the first day"""
        if df.empty:
            return df
        df = df.copy().drop(columns=["RowId", "ExpectedDividend", "SupervisionFlag"])
        df.sort_values(self._date_field_name, inplace=True)
        df = df.set_index(self._date_field_name)
        _close = self._get_first_close(df)
        for _col in df.columns:
            # NOTE: retain original adjclose, required for incremental updates
            if _col in [self._symbol_field_name, "adjclose", "change"]:
                continue
            if _col == "volume":
                df[_col] = df[_col] * _close
            else:
                df[_col] = df[_col] / _close
        return df.reset_index()
    
kwargs = {'qlib_data_1d_dir': qlib_data_1d_dir, 'old_qlib_data_dir': qlib_data_1d_dir}

yc = Normalize(
            source_dir=source_dir,
            target_dir=norm_dir,
            # normalize_class=YahooNormalizeCN1doffline,
            normalize_class=YahooNormalize1dExtendFixed,
            max_workers=2,
            date_field_name='date',
            symbol_field_name="instrument",
            **kwargs,
        )
# yc.normalize()

## problems
* 到底数据normalization 是怎么做的？ - 可以用中文数据分解每一步来验证，或者看看dump？
* def _get_calendar_list(self) 到底是什么作用？
* 

In [ ]:
# yc._target_dir
file_list = list(yc._source_dir.glob("*.csv"))
for file_path in file_list:
    # yc._executor(file_path)
    # print(yc._normalize_obj)
    file_path = Path(file_path)
    df = pd.read_csv(file_path)
    df = yc._normalize_obj.normalize(df)
    print(df)
    # yc._normalize_obj.print_all()
    # import inspect
    # print(yc._normalize_obj)
    # lines = inspect.getsource(yc._normalize_obj.normalize)
    # print(lines)
    if df is not None and not df.empty:
        if yc._end_date is not None:
            _mask = pd.to_datetime(df[self._date_field_name]) <= pd.Timestamp(yc._end_date)
            df = df[_mask]
        df.to_csv(yc._target_dir.joinpath(file_path.name), index=False)
        logger.info(f"writing {file_path.name} csv file {yc._target_dir.joinpath(file_path.name)}")
    else:
        logger.info(f"df is empty or none {file_path}, {yc._normalize_obj.normalize}")
    break

In [ ]:
# dump bin
csv_path=norm_dir
csv_files = sorted(csv_path.glob(f"*.csv") if csv_path.is_dir() else [csv_path])
if csv_files:
    max_workers = 10
    _dump = DumpDataUpdate(
        csv_path=norm_dir,
        qlib_dir=qlib_data_1d_dir,
        exclude_fields="instrument,date,ExpectedDividend,SupervisionFlag,RowId",
        max_workers=max_workers,
    )
    _dump.dump()

## qlib update_predictions

In [ ]:
import copy
import fire
import qlib
from qlib.constant import REG_CN
from qlib.model.trainer import task_train
from qlib.workflow.online.utils import OnlineToolR
from qlib.tests.config import CSI300_GBDT_TASK

task = copy.deepcopy(CSI300_GBDT_TASK)

task["record"] = {
    "class": "SignalRecord",
    "module_path": "qlib.workflow.record_temp",
}


class UpdatePredExample:
    def __init__(
        self, provider_uri="~/.qlib/qlib_data/cn_data", region=REG_CN, experiment_name="online_srv", task_config=task
    ):
        qlib.init(provider_uri=provider_uri, region=region)
        self.experiment_name = experiment_name
        self.online_tool = OnlineToolR(self.experiment_name)
        self.task_config = task_config

    def first_train(self):
        rec = task_train(self.task_config, experiment_name=self.experiment_name)
        self.online_tool.reset_online_tag(rec)  # set to online model

    def update_online_pred(self):
        self.online_tool.update_online_pred()

    def main(self):
        self.first1_train()
        self.update_online_pred()

# updater = UpdatePredExample(provider_uri="~/.qlib/qlib_data/cn_data", experiment_name="online_srv")


online_tool = OnlineToolR(backtest_experiment_name)
online_tool.update_online_pred(to_date=start_date, from_date=end_date, exp_name=backtest_experiment_name) # default update to file, so we have read it for submission.

experiment = R.get_exp(experiment_name=backtest_experiment_name)
recs = experiment.list_recorders()  # SignalRecord .load("pred.pkl")
recs

In [ ]:
experiment.get_recorder()
# list(recs.values())[0].load_object("pred.pkl")
# list(recs.values())[0].end_time

In [ ]:
# dir(list(recs.values())[0])

In [ ]:
# url used in this function https://www.szse.cn/api/report/exchange/onepersistenthour/monthList?month=2022-01&random=123455

from data_collector.utils import get_calendar_list
get_calendar_list()[-10:]
len(get_calendar_list())


In [ ]:
pd.Timestamp.now()

In [ ]:
import qlib
from qlib.data import D
# qlib.init(provider_uri="~/.qlib/qlib_data/cn_data")
qlib.init(provider_uri=qlib_data_1d_dir)
list(D.calendar(freq="day"))[-10:]
len(list(D.calendar(freq="day")))

In [ ]:
last_day = list(D.calendar(freq="day"))[-1]
print(last_day)
# i = 1
# while True:
#     day = last_day + pd.Timedelta(days=i)
#     if day < pd.Timestamp.now():
#         print(day)
#     else:
#         break
#     i += 1